In [1]:
""" pip install -qU \
    langchain==0.0.292 \
    openai==0.27.4 \
    datasets==2.10.1 \
    pinecone-client==2.2.4 \
    tiktoken==0.5.1
    """

' pip install -qU     langchain==0.0.292     openai==0.27.4     datasets==2.10.1     pinecone-client==2.2.4     tiktoken==0.5.1\n    '

In [2]:
import os

#os.environ["OPENAI_API_KEY"] = ""

In [1]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat = ChatOpenAI(
    openai_api_key = os.environ['OPENAI_API_KEY'],
    model='gpt-3.5-turbo'
)

/home/tema/10X/week7/Precision -RAG/Renv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

In [5]:
res = chat(messages)

print(res.content)

String theory is a theoretical framework in physics that attempts to reconcile quantum mechanics and general relativity. It posits that the fundamental building blocks of the universe are not particles, but rather tiny, vibrating strings. These strings can have different vibrational modes, which manifest as different particles when observed at different energy levels.

String theory suggests that there are additional spatial dimensions beyond the three we experience in our everyday lives. These extra dimensions are compactified and not directly observable at our scale. The theory also predicts the existence of multiple universes, known as the multiverse.

String theory has not yet been experimentally confirmed, and it remains a subject of active research and debate in the physics community. It is a highly complex and mathematical theory that requires a solid understanding of quantum mechanics and advanced mathematics to fully grasp.


# Automatic Evaluation Data Generation


##Evaluating RAG

In [7]:
chunk = 'There is suggestive evidence that photosynthetic organisms were present approximately 3.2 to 3.5 billion years ago, in the form of stromatolites, layered structures similar to forms that are produced by some modern cyanobacteria, as well as numerous microfossils that have been interpreted as arising from phototrophs (Des Marais, 2000).'


In [9]:
#Example QA generation prompt
from typing import Final
GOLD_STANDARD_QA_GENERATION_PROMPT: Final[
    str
] = "Given the following paragraph, please generate a single question and its corresponding answer.\n Question start with a 'Q:' and answers start with an 'A:'. The answer should be the next line after the question. Just output a question and answer according to the instruction."

In [10]:
question =  'When did photosynthetic organisms emerge?'

answer = 'Photosynthetic organisms emerged between 3.2 and 3.5 billion years ago.'

source = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2949000/'

wrong_answer = 'Photosynthetic organisms emerged between 3.2 and 2.4 billion years ago.'

wrong_source = 'https://en.wikipedia.org/wiki/History_of_Earth'

In [11]:
# Question passed into the RAG Chatbot
question = "When did photosynthetic organisms emerge?"

# This would actually be populated from calling our RAG system API
rag_api_response = {
    "sources": [
        "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2949000/"
    ],
    "chatbot_response": [
        {
            "message": "Photosynthetic organisms emerged between 3.2 and 3.5 billion years ago.",
        }
    ],
    "cost": "0.001662"
}

In [12]:
rag_source = rag_api_response['sources']
rag_answer = rag_api_response['chatbot_response'][0]['message']
rag_cost = float(rag_api_response['cost'])

faked_rag_response = [rag_answer, rag_source, rag_cost]
print(faked_rag_response)

['Photosynthetic organisms emerged between 3.2 and 3.5 billion years ago.', ['https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2949000/'], 0.001662]


In [13]:
# Metric Prompt for Evaluation
ACCURACY_PROMPT: Final[
    str
] = "Looking at the gold standard answer \n'{answer}'\n and the language model's answer \n'{faked_rag_response}'\n, would you state that the language model's answer is completely accurate?"

In [14]:
# Evaluate Answers
def evaluate_answers(metric_prompts : list[str]) -> list[str]:
    '''
    Main function to evaluate answers.
    Args:
        prompts (list[str]): Prompts to be evaluated
    Returns:
      (list[str]): A list with prompts evaluation results
    '''

    evaluations = []
    input_tokens = 0
    completion_tokens = 0
    for prompt in metric_prompts:
        text = prompt
        response = client.chat.completions.create(
            model="gpt-4",
            temperature=0,
            messages=[
                {
                    "role": "system",
                    "content": "You are an LLM evaluation assistant that will evaluate the given information. Evaluate the following results with 'Yes/No' followed by the 'reason' on a newline. 'Yes/No\nreason'. If you are unsure, please respond with 'I don't know.\nNo'.",
                },
                {"role": "user", "content": text},
            ],
        )
        input_tokens += response.usage.prompt_tokens
        completion_tokens += response.usage.completion_tokens
        evaluations.append(response.choices[0].message.content.strip())

    return evaluations, input_tokens, completion_tokens